In [4]:
import janitor
import pandas as pd
import re
from datetime import datetime


In [5]:

leads = pd.read_csv('data/all_time_leads.csv').clean_names()


selected_leads = leads[['lead_id','mobile_primary',
                        'business_phone_line_1','brizo_id', 
                        'created_date',
                        'first_mel_timestamp','latest_mql_timestamp',
                        'assigned_time_ae_',
                        'opportunity_created_date',
                        'closed_won_date',
                        'stage','unqualified_reason', 'closed_lost_reason']].drop_duplicates()


selected_leads['mobile_primary'] = selected_leads['mobile_primary'].astype(str)
selected_leads['business_phone_line_1'] = selected_leads['business_phone_line_1'].astype(str)

selected_leads['mobile_primary'] = selected_leads['mobile_primary'].str.replace(r'\D+', '', regex=True)
selected_leads['business_phone_line_1'] = selected_leads['business_phone_line_1'].str.replace(r'\D+', '', regex=True)



def clean_to_10(s):
    # 1) remove any non-digits
    digits = re.sub(r'\D+', '', str(s))
    # 2) if >10 digits and starts with "1", drop the leading "1"
    if len(digits) > 10 and digits.startswith('1'):
        digits = digits[1:]
    # 3) only keep if exactly 10 digits
    return digits if len(digits) == 10 else None

for col in ['mobile_primary', 'business_phone_line_1']:
    selected_leads[col] = selected_leads[col].apply(clean_to_10)


Change timestamp format into date

In [6]:
# date_cols = [
#     'created_date',
#     'first_mel_timestamp',
#     'latest_mql_timestamp',
#     'assigned_time_ae_',
#     'opportunity_created_date',
#     'closed_won_date'
# ]

# date_cols = selected_leads[date_cols]
# date_cols.to_csv('data/exported_data/date_cols.csv', index=False)

timestamp_col = [
    'first_mel_timestamp',
    'latest_mql_timestamp',
    'assigned_time_ae_',
]

for col in timestamp_col:
    selected_leads[col] = pd.to_datetime(selected_leads[col], errors='coerce').dt.date


/var/folders/8c/z2jj76jx2wq4dxt7jlmgxknm0000gn/T/ipykernel_2667/2072314949.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  selected_leads[col] = pd.to_datetime(selected_leads[col], errors='coerce').dt.date
/var/folders/8c/z2jj76jx2wq4dxt7jlmgxknm0000gn/T/ipykernel_2667/2072314949.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  selected_leads[col] = pd.to_datetime(selected_leads[col], errors='coerce').dt.date
/var/folders/8c/z2jj76jx2wq4dxt7jlmgxknm0000gn/T/ipykernel_2667/2072314949.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  selected_leads[col] = pd.to_da

In [7]:
# Sort the DataFrame by first_mel_timestamp (newest first)
selected_leads = selected_leads.sort_values(by='first_mel_timestamp', ascending=False)

# Assign row number within each ID group
selected_leads['row_num'] = selected_leads.groupby('brizo_id').cumcount() + 1

# Keep only the top row (like QUALIFY row_number = 1)
top_leads = selected_leads[selected_leads['row_num'] == 1].copy()


In [8]:
# inspect
print(top_leads.head(5))

               lead_id mobile_primary business_phone_line_1          brizo_id  \
11623  00QUo00000SrWhZ     6179975325            8574040199  c2638ca51cce596d   
20861  00QUo00000Sr1ac     7272398834            7274479663  a862fd2975ec7c44   
19238  00Q8b00001xKLPF     7049571547            7048228830  d615ce74fd940a54   
12144  00QUo00000SmMKi     7202263389            3032371133  62800acf4b06529e   
19845  00QUo00000STVhp     8599072053            8598232888  7aba6d8ee839fe7a   

      created_date first_mel_timestamp latest_mql_timestamp assigned_time_ae_  \
11623     4/4/2023          2025-07-08           2025-07-08        2025-07-08   
20861   10/31/2022          2025-07-07           2025-07-08        2025-07-07   
19238   10/31/2022          2025-07-07           2025-07-07        2025-07-07   
12144    3/22/2023          2025-07-06                  NaT               NaT   
19845   10/31/2022          2025-07-01           2025-07-01        2025-07-01   

      opportunity_created_

In [9]:
# save
top_leads.to_csv('data/exported_data/deduped_leads_long.csv', index=False)